# Create Agent using NVIDIA NIM

Here I build simple agent that call goweather.xyz API to measure temperature given city name by leveraging LangChain.

# Load Library

In [10]:
import os
from dotenv import load_dotenv
import requests

from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.tools import tool
from langchain.agents import create_agent

# Set Config

In [2]:
# load config inside .env
load_dotenv()

True

In [ ]:
# check NVIDIA_API_KEY
# os.environ["NVIDIA_API_KEY"]

# Define Model

In [4]:
# define model
llm = ChatNVIDIA(model="nvidia/llama-3.1-nemotron-nano-4b-v1.1", max_completion_tokens=419)

In [5]:
# all supported model, uncomment script below to check
# [model.id for model in llm.available_models if model.model_type]

In [6]:
# models for tool calling
[model for model in ChatNVIDIA.get_available_models() if model.supports_tools]

[Model(id='qwen/qwq-32b', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=None, supports_tools=True, supports_structured_output=False, supports_thinking=True, base_model=None),
 Model(id='nvidia/llama-3.1-nemotron-ultra-253b-v1', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=None, supports_tools=True, supports_structured_output=False, supports_thinking=True, base_model=None),
 Model(id='meta/llama-3.1-8b-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=None, supports_tools=True, supports_structured_output=True, supports_thinking=False, base_model=None),
 Model(id='qwen/qwen3-next-80b-a3b-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=None, supports_tools=True, supports_structured_output=True, supports_thinking=False, base_model=None),
 Model(id='nvidia/llama-3.1-nemotron-nano-4b-v1.1', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=None, supports_tools=True, supports_structured_output=Fa

In [7]:
@tool
def get_weather(query: str) -> str:
    """
    Use this tool to get current weather for a city
    """
    url = f"https://goweather.xyz/v2/weather/{query}"
    response = requests.get(url)
    temperature = response.json().get("temperature")
    if temperature:
        return f"Temperature : {temperature}"
    return "No data found."

tools = [get_weather]

# Define Agent

In [8]:
agent = create_agent(
    llm,
    tools=tools,
    system_prompt="You are a helpful AI assistant. Use tools when they are useful, especially for factual questions"
)

# Run Agent and Check Result

In [11]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "What's the weather in SF?"}]}
)

In [12]:
result['messages']

[HumanMessage(content="What's the weather in SF?", additional_kwargs={}, response_metadata={}, id='53be0302-7f88-4cda-b372-0570a6101799'),
 AIMessage(content='None', additional_kwargs={'tool_calls': [{'id': 'EhG6fPcSR', 'type': 'function', 'function': {'name': 'get_weather', 'arguments': '{"query": "SF"}'}}]}, response_metadata={'role': 'assistant', 'content': 'None', 'tool_calls': [{'id': 'EhG6fPcSR', 'type': 'function', 'function': {'name': 'get_weather', 'arguments': '{"query": "SF"}'}}], 'token_usage': {'prompt_tokens': 101, 'total_tokens': 123, 'completion_tokens': 22}, 'finish_reason': 'tool_calls', 'model_name': 'nvidia/llama-3.1-nemotron-nano-4b-v1.1'}, id='lc_run--2715ab5a-db2a-4957-99b2-b126a9f97f5e-0', tool_calls=[{'name': 'get_weather', 'args': {'query': 'SF'}, 'id': 'EhG6fPcSR', 'type': 'tool_call'}], usage_metadata={'input_tokens': 101, 'output_tokens': 22, 'total_tokens': 123}, role='assistant'),
 ToolMessage(content='Temperature : 5 °C', name='get_weather', id='3a0efa37

In [13]:
result['messages'][-1]

AIMessage(content='The current temperature in San Francisco is 5 °C. Stay warm!', additional_kwargs={}, response_metadata={'role': 'assistant', 'content': 'The current temperature in San Francisco is 5 °C. Stay warm!', 'token_usage': {'prompt_tokens': 172, 'total_tokens': 187, 'completion_tokens': 15}, 'finish_reason': 'stop', 'model_name': 'nvidia/llama-3.1-nemotron-nano-4b-v1.1'}, id='lc_run--c6ded72a-cc73-4032-9702-ccf0ac5d7a27-0', usage_metadata={'input_tokens': 172, 'output_tokens': 15, 'total_tokens': 187}, role='assistant')